In [1]:
import sqlite3
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
db_path = os.path.realpath("../DC4-data/firewall_data.db")
conn = sqlite3.connect(db_path)
print(conn.total_changes)

cur = conn.cursor()

0


In [45]:
def fill_df_nas(df, time_col, group_col, group_arr):
    """
    Expands dataframe to include all x-axis values for every group, and
    fills dataframes with NAs when there are no observations for the specified group.
    Useful for Plotly graphs in mode='lines+markers'
    :param: df: dataframe of interest
    :param: time_col: string name of column that contains time variable (or generally, the x variable)
    :param: group_col: string name of column that contains the groups to plot over different traces
    :param: group_arr: list or numpy array of all unique observations in df['group_col']
    :return: new dataframe
    """
    df = df.sort_values(by=[time_col])
    min_time = df[time_col].min()
    max_time = df[time_col].max()
    time_range = pd.date_range(min_time, max_time, freq='H')
    df_series = pd.Series(np.tile(group_arr, len(time_range)))
    df_idx_series = time_range \
        .repeat(len(group_arr))
    new_df = pd.DataFrame({time_col: df_idx_series,
                          group_col: df_series})
    df_with_nas = pd.merge(new_df, df, on=[time_col, group_col], how='left')
    return df_with_nas

## Exploratory analysis: column contents

In [19]:
# check colnames and types
cur.execute("PRAGMA table_info(data);")
print(cur.fetchall())

[(0, 'Date/time', 'date', 0, None, 0), (1, 'Syslog priority', 'text', 0, None, 0), (2, 'Operation', 'text', 0, None, 0), (3, 'Message code', 'text', 0, None, 0), (4, 'Protocol', 'text', 0, None, 0), (5, 'Source IP', 'text', 0, None, 0), (6, 'Destination IP', 'text', 0, None, 0), (7, 'Source hostname', 'text', 0, None, 0), (8, 'Destination hostname', 'text', 0, None, 0), (9, 'Source port', 'text', 0, None, 0), (10, 'Destination port', 'text', 0, None, 0), (11, 'Destination service', 'text', 0, None, 0), (12, 'Direction', 'text', 0, None, 0), (13, 'Connections built', 'integer', 0, None, 0), (14, 'Connections torn down', 'integer', 0, None, 0)]


In [7]:
# Get unique operations
q = "SELECT DISTINCT `Operation` \
         FROM data;"
cur.execute(q)
decisions = cur.fetchall()
decisions

[('Teardown',),
 ('Built',),
 ('(empty)',),
 ('Command executed',),
 ('Deny',),
 ('Deny by ACL',)]

## Overall traffic analysis

In [176]:
q = "SELECT strftime('%Y-%m-%d %H:00:00', `Date/time`) date_hour, COUNT(`Date/time`) n \
         FROM data \
         GROUP BY date_hour;"
traffic_per_hour = pd.read_sql_query(q, conn)
traffic_per_hour['date_hour'] = pd.to_datetime(traffic_per_hour['date_hour'])

In [181]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=traffic_per_hour['date_hour'],
    y=traffic_per_hour['n'],
    mode='lines+markers',
    connectgaps=False)
)
fig.update_layout(height=500,
                 width=800)
fig.show()

## Priority frequency over time analysis

In [34]:
# Get unique priorities
q = "SELECT DISTINCT `Syslog Priority` \
         FROM data;"
priorities_df = pd.read_sql_query(q, conn)
priorities = priorities_df['Syslog priority'].to_numpy()

In [25]:
q = "SELECT strftime('%Y-%m-%d %H:00:00', `Date/time`) date_hour, `Syslog Priority` priority, COUNT(`Date/time`) n \
         FROM data \
         GROUP BY date_hour, `Syslog Priority`;"
cxns_per_hour = pd.read_sql_query(q, conn)
cxns_per_hour['date_hour'] = pd.to_datetime(cxns_per_hour['date_hour'])

In [35]:
cxns_with_nas = fill_df_nas(cxns_per_hour, 'date_hour', 'priority', priorities)

['Info' 'Notice' 'Critical' 'Warning' 'Error']


In [42]:
visible = ['legendonly'] + [True]*(len(ports)-1)

fig = go.Figure()
for i in range(len(priorities)):
    expr = cxns_with_nas['priority'] == priorities[i]
    fig.add_trace(go.Scatter(
        x=cxns_with_nas[expr]['date_hour'],
        y=cxns_with_nas[expr]['n'],
        name=priorities[i],
        mode='lines+markers',
        connectgaps=False,
        visible=visible[i])
    )
fig.show()

## Port analysis

In [39]:
# Get top 10 source ports
cur.execute("SELECT `Source port`, COUNT(`Source port`) ct \
         FROM data \
         GROUP BY `Source port` \
         ORDER BY ct DESC \
         LIMIT 10;")
print(cur.fetchall())

[('6667', 32419), ('80', 9672), ('1822', 5562), ('1818', 5561), ('1779', 5552), ('1834', 5550), ('1819', 5547), ('1671', 5546), ('1743', 5545), ('1859', 5544)]


In [50]:
# Get top 10 destination ports
cur.execute("SELECT `Destination port`, COUNT(`Destination port`) ct \
         FROM data \
         GROUP BY `Destination port` \
         ORDER BY ct DESC \
         LIMIT 10;")
print(cur.fetchall())

[('80', 21331506), ('6667', 2329914), ('(empty)', 3204), ('21', 1825), ('53', 752), ('22', 538), ('1026', 199), ('137', 130), ('113', 62), ('1025', 52)]


In [196]:
cur.execute("SELECT strftime('%Y-%m-%d %H:00:00', `Date/time`) date_hour, COUNT(`Destination port`) n \
    FROM data \
    WHERE `Destination port` == '22' \
    GROUP BY date_hour")
print(cur.fetchall())

[('2012-04-05 20:00:00', 190), ('2012-04-05 21:00:00', 157), ('2012-04-05 22:00:00', 60), ('2012-04-05 23:00:00', 95), ('2012-04-06 00:00:00', 24), ('2012-04-06 01:00:00', 8), ('2012-04-06 03:00:00', 4)]


In [43]:
# prep port data
q = "SELECT strftime('%Y-%m-%d %H:00:00', `Date/time`) date_hour, \
         CASE \
             WHEN (`Destination port` = '80'\
                 OR `Destination port` = '6667' \
                 OR `Destination port` = '(empty)' \
                 OR `Destination port` = '21' \
                 OR `Destination port` = '53' \
                 OR `Destination port` = '22' \
                 OR `Destination port` = '137' \
                 OR `Destination port` = '113') \
                 THEN `Destination port` \
             ELSE 'OTHER' \
         END AS port_classification, \
         COUNT(`Date/time`) ct \
         FROM data \
         GROUP BY date_hour, port_classification;"

ports_per_hour = pd.read_sql_query(q, conn)
ports_per_hour['date_hour'] = pd.to_datetime(ports_per_hour['date_hour'])

In [44]:
ports = ['80', '6667', '(empty)', '21', '53', '22', '137', '113', 'OTHER']
ports_with_nas = fill_df_nas(ports_per_hour, 'date_hour', 'port_classification', ports)

['80', '6667', '(empty)', '21', '53', '22', '137', '113', 'OTHER']


In [182]:
fig = go.Figure()
# hide 80 and 6667
visible = ['legendonly']*2 + [True]*(len(ports)-2)
for i in range(len(ports)):
    expr = ports_with_nas['port_classification'] == ports[i]
    fig.add_trace(go.Scatter(
        x=ports_with_nas[expr]['date_hour'],
        y=ports_with_nas[expr]['ct'],
        name=ports[i],
        mode='lines+markers',
        connectgaps=False,
        visible=visible[i])
    )
fig.show()

## Parallel coordinates plot: Source IPs, destination ports

In [47]:
q = "SELECT `Source IP` source_ip, \
         CASE \
             WHEN (`Destination port` = '80'\
                 OR `Destination port` = '6667' \
                 OR `Destination port` = '(empty)' \
                 OR `Destination port` = '21' \
                 OR `Destination port` = '53' \
                 OR `Destination port` = '22' \
                 OR `Destination port` = '137' \
                 OR `Destination port` = '113') \
                 THEN `Destination port` \
             ELSE 'OTHER' \
         END AS port_classification, \
         COUNT(`Source IP`) n \
         FROM data \
         GROUP BY source_ip, port_classification;"
ports_cxns_per_ip = pd.read_sql_query(q, conn)

In [173]:
ports_cxns_per_ip_pivot = ports_cxns_per_ip.pivot(index='source_ip',columns='port_classification', values='n')
# add dummy continuous variable for pcoords color
ports_cxns_per_ip_pivot["dummy"] = range(len(ports_cxns_per_ip_pivot.index))

In [162]:
def create_dims(df, idx):
    dims = []
    # todo: add default rangesliders?
    for g in df.columns:
        if g != idx:
            d = {}
            d["range"] = [0, df[g].max()]
            d["label"] = g
            d["values"] = df[g]
            dims.append(d)
    return dims

In [174]:
dims = create_dims(ports_cxns_per_ip_pivot, "dummy")

In [175]:
fig = go.Figure(data=go.Parcoords(
        line = dict(color = ports_cxns_per_ip_pivot["dummy"]),
        dimensions = list(dims)
    )
)
# fig.update_layout(height=350, width=800)
fig.show()

In [205]:
cur.execute("SELECT * \
         FROM data \
         WHERE `Syslog priority` = 'Critical';")
print(cur.fetchone())

('2012-04-05 20:25:35', 'Critical', 'Deny', 'ASA-2-106001', 'TCP', '10.32.5.56', '10.32.0.1', '(empty)', '(empty)', '6667', '1688', '1688_tcp', 'inbound', 0, 0)


In [183]:
cur.close()
conn.close()